<a href="https://colab.research.google.com/github/ikhdaaakmalia/cloud/blob/main/131_FinancialTransactionDatasetAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Financial Transactions Dataset: Analytics**

In [ ]:
#install library di kaggle
!pip install kaggle
!pip install pandas

In [ ]:
#upload file kaggle.json
from google.colab import files
files.upload()

#simpan API key dan atur izin akses
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
#download dataset dari kaggle
!kaggle datasets download -d computingvictor/transactions-fraud-datasets

Dataset URL: https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets
License(s): apache-2.0
 80% 278M/348M [00:00<00:00, 640MB/s] 
100% 348M/348M [00:00<00:00, 635MB/s]


In [ ]:
!unzip transactions-fraud-datasets.zip

Archive:  transactions-fraud-datasets.zip
  inflating: cards_data.csv          
  inflating: mcc_codes.json          
  inflating: train_fraud_labels.json  
  inflating: transactions_data.csv   
  inflating: users_data.csv          


Users Data Analys

In [ ]:
#fungsi map reduce adalah untuk mengelompokan data

In [ ]:
from collections import defaultdict
import pandas as pd

#muat data
users_df = pd.read_csv('users_data.csv')

#=== MAP ===
mapped_counts = [] #utk menghitung jumlah pengguna per gender
mapped_scores = [] #utk menghitung skor kredit per gender

for index, row in users_df.iterrows():
  gender = row['gender']
  credit_score = row['credit_score']

  mapped_counts.append((gender, 1))
  mapped_scores.append((gender, credit_score))

#=== SHUFFLE & GROUP ===
grouped_counts = defaultdict(list)
grouped_scores = defaultdict(list)

for gender, count in mapped_counts:
  grouped_counts[gender].append(count)

for gender, score in mapped_scores:
  grouped_scores[gender].append(score)

#=== REDUCE ===
reduced_counts = {gender: sum(counts) for gender, counts in grouped_counts.items()}
reduced_avg_scores = {gender: sum(scores)/len(scores) for gender, scores in grouped_scores.items()}

#=== OUTPUT ===
print("jumlah pengguna per gender: ")
for gender, count in reduced_counts.items():
  print(f"{gender}: {count} orang")

print("\nrata rata skor kredit per gender: ")
for gender, avg in reduced_avg_scores.items():
  print(f"{gender}: {round(avg, 2)}")

jumlah pengguna per gender: 
Female: 1016 orang
Male: 984 orang

rata rata skor kredit per gender: 
Female: 709.17
Male: 710.32


Cards Data Analys

In [ ]:
from collections import defaultdict
import pandas as pd

# Load data kartu
cards_df = pd.read_csv("cards_data.csv")

# Bersihkan kolom credit_limit ($ ke float)
cards_df['credit_limit'] = cards_df['credit_limit'].replace('[\$,]', '', regex=True).astype(float)

# 1. Jumlah kartu per jenis
mapped_type = [(row['card_type'], 1) for _, row in cards_df.iterrows()]
grouped_type = defaultdict(list)
for k, v in mapped_type:
    grouped_type[k].append(v)
reduced_type = {k: sum(v) for k, v in grouped_type.items()}


# 2. Rata-rata limit kredit per card_type
mapped_limit = [(row['card_type'], row['credit_limit']) for _, row in cards_df.iterrows()]
grouped_limit = defaultdict(list)
for k, v in mapped_limit:
    grouped_limit[k].append(v)
reduced_avg_limit = {k: sum(v)/len(v) for k, v in grouped_limit.items()}


# 3. Jumlah kartu bocor (dark web exposure)
mapped_darkweb = [(row['card_on_dark_web'], 1) for _, row in cards_df.iterrows()]
grouped_darkweb = defaultdict(list)
for k, v in mapped_darkweb:
    grouped_darkweb[k].append(v)
reduced_darkweb = {k: sum(v) for k, v in grouped_darkweb.items()}

# Tampilkan hasil analisis
print("Jumlah kartu per jenis:")
for k, v in reduced_type.items():
    print(f"- {k}: {v} kartu")

print("\n Rata-rata credit limit per jenis:")
for k, v in reduced_avg_limit.items():
    print(f"- {k}: ${round(v, 2)}")

print("\n Status kebocoran (dark web):")
for k, v in reduced_darkweb.items():
    print(f"- {k}: {v} kartu")

Jumlah kartu per jenis:
- Debit: 3511 kartu
- Credit: 2057 kartu
- Debit (Prepaid): 578 kartu

 Rata-rata credit limit per jenis:
- Debit: $18557.89
- Credit: $11174.38
- Debit (Prepaid): $64.45

 Status kebocoran (dark web):
- No: 6146 kartu


Transaction Data Analys